<a href="https://colab.research.google.com/github/ZhiquanW/2017-MCM/blob/master/CUDA_Template(testing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Draft Notebook

## Configure CUDA Environment

In [2]:
# check nvidia cuda envionment
!nvcc --version
# install juypter cuda plugin
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
# load plugin
%load_ext nvcc_plugin

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-o77xxblz
  Stored in directory: /tmp/pip-ephem-wheel-cache-juqw6yei/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


## Pull Source Codes or Repository from Github

### Configure Local Git Environment

In [43]:
# check git version
! git --version
# clone repository from github
! git clone https://github.com/ZhiquanW/Learn-CUDA-Repository.git


git version 2.17.1
Cloning into 'Learn-CUDA-Repository'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 149 (delta 28), reused 134 (delta 17), pack-reused 0
Receiving objects: 100% (149/149), 1.76 MiB | 11.15 MiB/s, done.
Resolving deltas: 100% (28/28), done.


### Change to Work Directory

In [20]:
# show current working space
! ls

exe  hello.cu  Learn-CUDA-Repository  sample_data  src


In [0]:
# change to new work directory
! cd Learn-CUDA-Repository/
! mkdir new_dir
! cd new_dir

## Compile and Run CUDA Program

In [0]:
# compile codes with nvcc
! nvccc -o main.cu exe

In [0]:
# run with nvprofile
! nvprof ./exe

In [0]:
! ls
% cd Learn-CUDA-Repository/HelloCuda/
! ls
! cat hello.cu

In [9]:
# load files from local
from google.colab import files
files.upload()

Saving hello.cu to hello.cu


{'hello.cu': b'#include <math.h>\r\n#include <iostream>\r\n\r\n__global__ void add(int, float *, float *y);\r\nvoid hello_cuda();\r\n\r\nint main(void) {\r\n  hello_cuda();\r\n  return 0;\r\n}\r\n// Kernel function to add the elements of two arrays\r\n__global__ void add(int n, float *x, float *y) {\r\n  for (int i = 0; i < n; ++i) {\r\n    y[i] = x[i] + y[i];\r\n  }\r\n}\r\n\r\nvoid hello_cuda() {\r\n  int N = 1 << 20;\r\n  float *x, *y;\r\n\r\n  // Allocate Unified Memory \xe2\x80\x93 accessible from CPU or GPU\r\n  cudaMallocManaged(&x, N * sizeof(float));\r\n  cudaMallocManaged(&y, N * sizeof(float));\r\n  // initialize x and y arrays on the host\r\n  for (int i = 0; i < N; i++) {\r\n    x[i] = 1.0f;\r\n    y[i] = 2.0f;\r\n  }\r\n\r\n  // Run kernel on 1M elements on the GPU\r\n  add<<<1, 1>>>(N, x, y);\r\n\r\n  // Wait for GPU to finish before accessing on host\r\n  cudaDeviceSynchronize();\r\n\r\n  // Check for errors (all values should be 3.0f)\r\n  float maxError = 0.0f;\r\n  f

In [14]:
! nvcc hello.cu -o exe
! nvprof ./exe

==585== NVPROF is profiling process 585, command: ./exe
Max error: 8
1048576
==585== Profiling application: ./exe
==585== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  447.13ms         1  447.13ms  447.13ms  447.13ms  add(int, float*, float*)
      API calls:   69.71%  447.18ms         1  447.18ms  447.18ms  447.18ms  cudaDeviceSynchronize
                   29.63%  190.03ms         2  95.015ms  875.74us  189.15ms  cudaMallocManaged
                    0.33%  2.1343ms         1  2.1343ms  2.1343ms  2.1343ms  cudaLaunchKernel
                    0.24%  1.5322ms         2  766.08us  760.56us  771.61us  cudaFree
                    0.05%  337.83us        96  3.5190us     161ns  148.38us  cuDeviceGetAttribute
                    0.03%  187.79us         1  187.79us  187.79us  187.79us  cuDeviceTotalMem
                    0.01%  38.783us         1  38.783us  38.783us  38.783us  cuDeviceGetName
                 

In [13]:
%%writefile hello.cu
#include <math.h>
#include <iostream>

__global__ void add(int, float *, float *y);
void hello_cuda();

int main(void) {
  hello_cuda();
  return 0;
}
// Kernel function to add the elements of two arrays
__global__ void add(int n, float *x, float *y) {
  for (int i = 0; i < n; ++i) {
    y[i] = x[i] * y[i];
  }
}

void hello_cuda() {
  int N = 1 << 20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N * sizeof(float));
  cudaMallocManaged(&y, N * sizeof(float));
  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++) maxError = fmax(maxError, fabs(y[i] - 10.0f));
  std::cout << "Max error: " << maxError << std::endl;
  std::cout << N << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);
}

Overwriting hello.cu
